# Necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS
from scipy.interpolate import RBFInterpolator
from pandas._libs.tslibs.parsing import guess_datetime_format
from scipy.interpolate import Rbf

2023-05-11 15:14:18.078146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-11 15:14:18.078165: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df_for=pd.read_csv("4_df_cleaned_for_outlier_removal.csv")
df_for=df_for.drop(["Unnamed: 0"],axis=1)
df_for

,PatientID,DateAdded_New,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1725661,10210244,2023-01-05,1,0.94,0.84,87.58,34.264272,3.0,0.7896,M,White,Non-Hispanic
1725662,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1725663,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1725664,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


In [3]:
df_for.rename(columns = {'DateAdded_New':'DateAddedEST'}, inplace = True)
df_for

,PatientID,DateAddedEST,WoundNumber,Length,Width,Age,BMI,SmokingStatus,Area,PatientGender_new,RaceCodeList_new,patientEthnicity_new
0,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
1,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
2,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
3,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
4,5978401,2016-10-26,1,2.00,2.00,71.81,27.500000,4.0,4.0000,M,White,Non-Hispanic
...,...,...,...,...,...,...,...,...,...,...,...,...
1725661,10210244,2023-01-05,1,0.94,0.84,87.58,34.264272,3.0,0.7896,M,White,Non-Hispanic
1725662,10223609,2023-01-09,1,1.74,3.35,56.66,34.264272,4.0,5.8290,F,White,Hispanic
1725663,10223609,2023-01-09,1,1.81,3.39,56.66,34.264272,4.0,6.1359,F,White,Hispanic
1725664,10219057,2023-01-09,2,4.05,6.11,73.30,44.600000,3.0,24.7455,M,White,Non-Hispanic


## Defining the necessary functions

In [4]:
def remove_wrong_dates(df_patient_1_wound):
    df_patient_1_wound=df_patient_1_wound.sort_values(by=['DateAddedEST'])
    date_format_1 = guess_datetime_format(df_patient_1_wound["DateAddedEST"][df_patient_1_wound.shape[0]-1])
    date_format_2 = guess_datetime_format(df_patient_1_wound["DateAddedEST"][0])
    a0 = datetime.strptime(df_patient_1_wound["DateAddedEST"][df_patient_1_wound.shape[0]-1], date_format_1)
    b0 = datetime.strptime(df_patient_1_wound["DateAddedEST"][0], date_format_2)
    delta = a0 - b0

    if delta.days>800:
        date_format_1 = guess_datetime_format(df_patient_1_wound["DateAddedEST"][df_patient_1_wound.shape[0]-1])
        date_format_2 = guess_datetime_format(df_patient_1_wound["DateAddedEST"][0])
        a1 = datetime.strptime(df_patient_1_wound["DateAddedEST"][df_patient_1_wound.shape[0]-2], date_format_1)
        b1 = datetime.strptime(df_patient_1_wound["DateAddedEST"][1], date_format_2)
        #print((a0-a1).days)
        #print((b1-b0).days)
        #print(b1)
        #print(b0)
        if (a0-a1).days>500:
            df_patient_1_wound=df_patient_1_wound.drop(df_patient_1_wound.tail(1).index)
        elif (b1-b0).days>500:
            df_patient_1_wound=df_patient_1_wound.drop(df_patient_1_wound.head(1).index)            
    return df_patient_1_wound

In [5]:
def get_date_and_area(df_patient_1_wound):
    df_patient_1_wound['DateAddedEST'] = pd.to_datetime(df_patient_1_wound.DateAddedEST,utc=True)
    df_patient_1_wound['DateAddedEST'] = df_patient_1_wound['DateAddedEST'].dt.strftime('%Y/%m/%d')
    #display(df_patient_1_wound)
    area=df_patient_1_wound["Length"].values*df_patient_1_wound["Width"].values
    area=area.tolist()
    date=df_patient_1_wound["DateAddedEST"].values
    date=date.tolist()
    return (area,date)

In [6]:
def indexing_dataframe(date,area):
    df=pd.DataFrame()
    df["Date"]=date
    df["Area"]=area
    df=df.drop_duplicates(subset=['Date'], keep='first')
    df=df.reset_index()
    df=df.drop(["index"],axis=1)
    

    dff=df.set_index('Date')
    #print("dff shape",dff.shape[0])
    return (dff,df)

In [ ]:
list_patients=df_for["PatientID"].unique().tolist()
list_wounds=[]
wound=0
df_merged=pd.DataFrame()
wounds_with_outliers=[]

for i in range(len(list_patients)):
#     if i>4:
#         break
#     if i==6082:
    patient_id=list_patients[i]
    df_indivi_patient=df_for.query("PatientID==@patient_id")
    #display(df_indivi_patient)
    list_wounds=df_indivi_patient["WoundNumber"].unique().tolist()
    for j in range(len(list_wounds)):
        print("Patient:",i+1," Wound:",j+1)
        list_wounds.append(wound)
        gender=df_indivi_patient["PatientGender_new"].unique()
        race=df_indivi_patient["RaceCodeList_new"].unique()
        ethnicity=df_indivi_patient["patientEthnicity_new"].unique()
        age=df_indivi_patient["Age"].unique()

        
        wound_number=list_wounds[j]
        df_indivi_patient_indivi_wound=df_indivi_patient.query("WoundNumber==@wound_number")
        #display(df_indivi_patient_indivi_wound)

        df_patient_1_wound=df_indivi_patient_indivi_wound

        df_patient_1_wound=df_patient_1_wound.sort_values(by=['DateAddedEST'])
        df_patient_1_wound=df_patient_1_wound.reset_index()
        df_patient_1_wound=df_patient_1_wound.drop(["index"],axis=1)
        #display(df_patient_1_wound)
        df_patient_1_wound=df_patient_1_wound.dropna()
        df_patient_1_wound=df_patient_1_wound.reset_index()
        df_patient_1_wound=df_patient_1_wound.drop(["index"],axis=1)


        df_patient_1_wound=df_patient_1_wound.replace(regex=["-04:00"],value="")
        df_patient_1_wound=df_patient_1_wound.replace(regex=["-05:00"],value="")

        if df_patient_1_wound.shape[0]>1:

            #df_patient_1_wound=remove_wrong_dates(df_patient_1_wound)

    #         if df_patient_1_wound.dropna().shape[0]>1:

    #             w_length=pd.Series(df_patient_1_wound["Length"].values)
    #             y = w_length.dropna().values
    #             x=w_length.dropna().index.values
    #             cs = CS(x,y)
    #             #rbf = RBFInterpolator(x,y,kernel='multiquadric')
    #             x1=w_length.index.values
    #             y1=cs(w_length.index.values)
    #             df_patient_1_wound["Length"]=y1


    #             w_width=pd.Series(df_patient_1_wound["Width"].values)
    #             y = w_width.dropna().values
    #             x=w_width.dropna().index.values
    #             cs = CS(x,y)
    #             #rbf = RBFInterpolator(x,y,kernel='multiquadric')
    #             x1=w_width.index.values
    #             y1=cs(w_width.index.values)
    #             df_patient_1_wound["Width"]=y1

    #             w_depth=pd.Series(df_patient_1_wound["Depth"].values)
    #             y = w_depth.dropna().values
    #             x=w_depth.dropna().index.values
    #             #rbf = RBFInterpolator(x,y,kernel='multiquadric')
    #             cs = CS(x,y)
    #             x1=w_depth.index.values
    #             y1=cs(w_depth.index.values)
    #             df_patient_1_wound["Depth"]=y1

    #             #display(df_patient_1_wound)
    #         else:


    #             df_patient_1_wound['Length'].fillna((df_patient_1_wound['Length'].mean()), inplace=True)
    #             df_patient_1_wound['Width'].fillna((df_patient_1_wound['Width'].mean()), inplace=True)
    #             df_patient_1_wound['Depth'].fillna((df_patient_1_wound['Depth'].mean()), inplace=True)
    #             #display(df_patient_1_wound)


            #Do the interpolation for all dates
            area, date= get_date_and_area(df_patient_1_wound)    
            #print(area,date)
            dff,df=indexing_dataframe(date,area)
            #display(df)
            #print("dff shape",dff.shape[0])
            if df.shape[0]>1:
                #display(df)
                wound=wound+1
                print("Number of visit days before interpolation is:",df.shape[0])
            
                # IQR
                Q1 = np.percentile(df['Area'], 25,
                                   interpolation = 'midpoint')

                Q3 = np.percentile(df['Area'], 75,
                                   interpolation = 'midpoint')
                IQR = Q3 - Q1

                print("Old Shape: ", df.shape)

                # Upper bound
                upper=Q3+2*IQR
                # Lower bound
                lower=Q1-2*IQR

                print("New Shape: ", df.shape)

                list_index=[]
                df=df.reset_index()
                for k in range(list(df.index.values)[0],list(df.index.values)[len(list(df.index.values))-1]):
                    #print(i)
                    if (df["Area"][k]>upper or  df["Area"][k]<lower):
                        list_index.append(k)
                        wounds_with_outliers.append(wound)
                df=df.drop(list_index,axis=0)
                
                
                
                
                df=df.sort_values(by=['Date'])

                #display(df)


                df=df.reset_index()
                df=df.drop(["index"],axis=1)
                #wound=wound+1
                df["Wound"]=wound
                df["Gender"]=gender[0]
                df["Race"]=race[0]
                df["Ethinicity"]=ethnicity[0]
                df["Age"]=age.mean()

                #display(df)

#                 dff=dff.sort_values(by=['Date'])

#                 s=interpolating_dates(dff,df)
#                 df_interpolated=interpolating_areas(s)


#                 df=s.to_frame().reset_index()

#                 df_interpolated["Wound"]=wound
#                 df_interpolated["Gender"]=gender[0]
#                 df_interpolated["Race"]=race[0]
#                 df_interpolated["Ethinicity"]=ethnicity[0]

                df_merged = pd.concat([df_merged, df], ignore_index=True, sort=False)

#                 #Plotting the interpolation

#                 x=df.index.values
#                 y=df.Area.values

#                 xnew=df_interpolated.index.values
#                 ynew=df_interpolated.Area.values
#                 plt.plot(x, y, 'o', xnew, ynew, '-')
#                 plt.plot(x,y,'o')
#                 plt.show()
#                 if df_interpolated.shape[0]==587:
#                     break
#     if df_interpolated.shape[0]==587:
#         break

                #display(df_interpolated)


In [10]:
df_merged

,level_0,Date,Area,Wound,Gender,Race,Ethinicity,Age
0,0,2016/10/26,4.0000,1,M,White,Non-Hispanic,71.969167
1,1,2016/11/01,4.0000,1,M,White,Non-Hispanic,71.969167
2,2,2016/11/02,4.0000,1,M,White,Non-Hispanic,71.969167
3,3,2016/11/07,7.5000,1,M,White,Non-Hispanic,71.969167
4,4,2016/11/09,4.0000,1,M,White,Non-Hispanic,71.969167
...,...,...,...,...,...,...,...,...
159041,0,2023/01/03,2.7064,17884,F,White,Non-Hispanic,71.230000
159042,1,2023/01/09,2.7064,17884,F,White,Non-Hispanic,71.230000
159043,1,2023/01/10,8.9401,17885,M,White,Non-Hispanic,68.370000
159044,1,2023/01/10,5.2528,17886,M,White,Non-Hispanic,68.370000


## So we have removed the outlier assessment values from each wound and got the cleaned dataframe

In [11]:
len(wounds_with_outliers)

8608

In [12]:
df_merged.to_csv("5_df_for_interpolation_may11.csv")